In [2]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import math

In [3]:
train = pd.read_csv('train.csv')
#test = pd.read_csv('test.csv')
validation = pd.read_csv('new_validation.csv')

In [4]:
XG = pd.read_csv('XG_validation.csv')

In [18]:
validation_o =  pd.read_csv('validation.csv')

In [3]:
Ada = pd.read_csv('Ada_validation.csv')

In [46]:
# c = 6
# lambdas = 0.000025

In [40]:
c =  np.linspace(6,6 , 1)
lambdas = np.linspace(0.000025,  0.0000025, 1)

In [6]:
avgCTR = 0.0007375623256619447

<font face="微软雅黑" size=6 color=#000000> Squared Bidding

In [100]:
def Non_LinearStrategy(pCTR, base_bid,avgCTR,m,n):
    avgCTR = np.repeat(avgCTR*100, pCTR.shape[0])
    base_bid =  np.repeat(base_bid, pCTR.shape[0])
    m= np.repeat(m, pCTR.shape[0])
    n= np.repeat(n, pCTR.shape[0])
    bidprices  =  base_bid  * ((pCTR['pCTR']/ avgCTR)**2) + m* (pCTR['pCTR']/ avgCTR) + n
    return bidprices

<font face="微软雅黑" size=6 color=#000000> Expoential Bidding 

In [9]:
def Exp(pCTR, base_bid,avgCTR):
    avgCTR = np.repeat(avgCTR*100, pCTR.shape[0])
    base_bid =  np.repeat(base_bid, pCTR.shape[0])
    bidprices  = base_bid  * np.exp(pCTR['pCTR']/ avgCTR)
    return bidprices

In [149]:
#lambda_range =np.linspaces (math.exp(-10),     )

<font face="微软雅黑" size=6 color=#000000> ORTB1

In [4]:

def Optimal_RealTime_Bidding_1(pCTR, c, lambdas):
    c= np.repeat(c, pCTR.shape[0])
    lambdas = np.repeat(lambdas, pCTR.shape[0])
    bidprices = np.sqrt( (c/lambdas * pCTR['pCTR'] )+c**2) - c
    return bidprices

<font face="微软雅黑" size=6 color=#000000> ORTB2

In [25]:

def Optimal_RealTime_Bidding_2(pCTR, c, lambdas):
    c= np.repeat(c, pCTR.shape[0])
    lambdas = np.repeat(lambdas, pCTR.shape[0])
    a =  ((pCTR['pCTR'] + np.sqrt( (c**2)*(lambdas**2) + lambdas**2))/(c*lambdas))**(1/3)  
    b = ((c*lambdas)/ (pCTR['pCTR'] + np.sqrt((c**2)*(lambdas**2) + lambdas**2)))**(1/3)  
    bidprices =c*(a - b)
    return bidprices

<font face="微软雅黑" size=6 color=#000000> Strategy Evaluation 

In [101]:
def Evaluate(validation, pre_bidprices):
    
    bidprice = pre_bidprices
    budget = 6250000

    #find who wins the bid
    payprice = validation['payprice'].values
    win_bid = bidprice>payprice

    spend = np.cumsum(payprice*win_bid)
    enough_budget = spend<budget


    click =  np.sum(validation['click'].values*win_bid*enough_budget)
    impression = np.sum(win_bid*enough_budget)
    spend = np.max(spend*enough_budget)
    ctr = click/impression*100
    cpm = spend/impression*1000
    cpc = spend/click


    return [impression,click, ctr, spend, cpm, cpc]

<font face="微软雅黑" size=6 color=#000000> Tuning Parameter for Square 

In [102]:
basebids = np.linspace(100, 100, 1) 

In [103]:
range_m = np.linspace(120, 130, 15) 

In [104]:
range_n =np.linspace(2, 10, 10) 

In [105]:
k=0
results  = pd.DataFrame(columns = ["basebid","impression","click","ctr","spend","cpm","cpc","m","n"])     

for basebid in basebids:
    for m in range_m:
        for n in range_n:
            pre_bidprices = Non_LinearStrategy(XG, basebid,avgCTR,m,n)
            impression, click, ctr, spend, cpm, cpc = Evaluate(validation, pre_bidprices)
            results.loc[k]  = np.round(basebid,5) , impression, click, ctr, spend, cpm, cpc, m, n
            k = k+1

/Users/sue/anaconda3/lib/python3.6/site-packages/pandas/core/computation/expressions.py:180: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  .format(op=op_str, alt_op=unsupported[op_str]))


In [106]:
results.click.max()

144.0

In [76]:
best = results.loc[(results['click'] == 171)]

In [77]:
best

,basebid,impression,click,ctr,spend,cpm,cpc,m,n
26,100.0,126171.0,171.0,0.135530,6249950.0,49535.550959,36549.415205,121.428571,7.333333
45,100.0,125001.0,171.0,0.136799,6224354.0,49794.433645,36399.730994,122.857143,6.444444
55,100.0,125317.0,171.0,0.136454,6244546.0,49829.999122,36517.812865,123.571429,6.444444
74,100.0,124033.0,171.0,0.137867,6210086.0,50068.014158,36316.292398,125.000000,5.555556
84,100.0,124374.0,171.0,0.137489,6232575.0,50111.558686,36447.807018,125.714286,5.555556
94,100.0,124641.0,171.0,0.137194,6249966.0,50143.740824,36549.508772,126.428571,5.555556
103,100.0,123054.0,171.0,0.138963,6197540.0,50364.392868,36242.923977,127.142857,4.666667
113,100.0,123388.0,171.0,0.138587,6217980.0,50393.717379,36362.456140,127.857143,4.666667
122,100.0,121737.0,171.0,0.140467,6162007.0,50617.371875,36035.128655,128.571429,3.777778
123,100.0,123750.0,171.0,0.138182,6241167.0,50433.672727,36498.052632,128.571429,4.666667


In [94]:
basebids = np.linspace(400, 420, 10) 

In [95]:
k=0
results  = pd.DataFrame(columns = ["basebid","impression","click","ctr","spend","cpm","cpc"])     

for basebid in basebids:
    pre_bidprices = Non_LinearStrategy(XG, basebid,avgCTR)
    impression, click, ctr, spend, cpm, cpc = Evaluate(validation, pre_bidprices)
    results.loc[k]  = np.round(basebid,5) , impression, click, ctr, spend, cpm, cpc
    k = k+1

/Users/sue/anaconda3/lib/python3.6/site-packages/pandas/core/computation/expressions.py:180: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  .format(op=op_str, alt_op=unsupported[op_str]))


In [96]:
results.click.max()

168.0

In [97]:
results

,basebid,impression,click,ctr,spend,cpm,cpc
0,400.00000,99479.0,168.0,0.168880,6138300.0,61704.480343,36537.500000
1,402.22222,99751.0,168.0,0.168419,6157165.0,61725.346112,36649.791667
2,404.44444,100030.0,168.0,0.167950,6177625.0,61757.722683,36771.577381
3,406.66667,100290.0,168.0,0.167514,6194847.0,61769.338917,36874.089286
4,408.88889,100546.0,168.0,0.167088,6211353.0,61776.231775,36972.339286
5,411.11111,100838.0,168.0,0.166604,6232792.0,61809.952597,37099.952381
6,413.33333,101092.0,168.0,0.166185,6249881.0,61823.695248,37201.672619
7,415.55556,101061.0,167.0,0.165247,6249958.0,61843.421300,37424.898204
8,417.77778,101025.0,167.0,0.165306,6249954.0,61865.419451,37424.874251
9,420.00000,100983.0,166.0,0.164384,6249903.0,61890.644960,37650.018072


<font face="微软雅黑" size=6 color=#000000> Tuning Parameter for Exponential

In [18]:
k=0
results  = pd.DataFrame(columns = ["basebid","impression","click","ctr","spend","cpm","cpc"])     

for basebid in basebids:
    pre_bidprices = Exp(XG, basebid,avgCTR)
    impression, click, ctr, spend, cpm, cpc = Evaluate(validation, pre_bidprices)
    results.loc[k]  = np.round(basebid,5) , impression, click, ctr, spend, cpm, cpc
    k = k+1

In [19]:
results.click.max()

161.0

In [20]:
results

,basebid,impression,click,ctr,spend,cpm,cpc
0,20.00000,76263.0,124.0,0.16260,2079856.0,27272.15032,16773.03
1,23.33333,86822.0,129.0,0.14858,2519716.0,29021.63046,19532.68
2,26.66667,95980.0,132.0,0.13753,2967437.0,30917.24318,22480.58
3,30.00000,105449.0,139.0,0.13182,3451408.0,32730.59014,24830.27
4,33.33333,114634.0,145.0,0.12649,3962357.0,34565.28604,27326.60
5,36.66667,124218.0,155.0,0.12478,4525763.0,36434.03532,29198.47
6,40.00000,134894.0,157.0,0.11639,5171415.0,38336.87933,32938.95
7,43.33333,145988.0,161.0,0.11028,5859432.0,40136.39477,36393.99
8,46.66667,149173.0,147.0,0.09854,6249999.0,41897.65574,42517.00
9,50.00000,143231.0,140.0,0.09774,6249999.0,43635.79812,44642.85


In [21]:
best = results.loc[(results['click'] == 161)]

In [22]:
best

,basebid,impression,click,ctr,spend,cpm,cpc
7,43.33333,145988.0,161.0,0.11028,5859432.0,40136.39477,36393.99


<font face="微软雅黑" size=6 color=#000000> Tuning Parameter for ORTB1

In [53]:
# c =6 , lambda = 0.000025
c =  np.linspace(1, 10, 10)
lambdas = np.linspace(0.000024,  0.000024, 1)

In [54]:
k=0
results  = pd.DataFrame(columns = ["impression","click","ctr","spend","cpm","cpc","c","lambdas"])     

for pram_c in c:
    for lamb in lambdas:
        pre_bidprices = Optimal_RealTime_Bidding_1(XG, pram_c, lamb)
        impression, click, ctr, spend, cpm, cpc = Evaluate(validation, pre_bidprices)
        results.loc[k]  = impression, click, ctr, spend, cpm, cpc, pram_c, lamb
        k = k+1

In [55]:
results.click.max()

163.0

In [56]:
results

,impression,click,ctr,spend,cpm,cpc,c,lambdas
0,61454.0,97.0,0.15784,1366459.0,22235.47694,14087.21,1.0,0.000024
1,91550.0,123.0,0.13435,2670193.0,29166.49918,21708.89,2.0,0.000024
2,111722.0,140.0,0.12531,3766379.0,33712.06208,26902.71,3.0,0.000024
3,126950.0,145.0,0.11422,4718811.0,37170.62623,32543.52,4.0,0.000024
4,139087.0,154.0,0.11072,5541850.0,39844.48583,35986.04,5.0,0.000024
5,148827.0,163.0,0.10952,6233976.0,41887.39946,38245.25,6.0,0.000024
6,142755.0,145.0,0.10157,6249999.0,43781.29663,43103.44,7.0,0.000024
7,137834.0,140.0,0.10157,6249999.0,45344.39253,44642.85,8.0,0.000024
8,134042.0,134.0,0.09997,6249999.0,46627.16910,46641.78,9.0,0.000024
9,130601.0,127.0,0.09724,6249999.0,47855.67492,49212.59,10.0,0.000024


<font face="微软雅黑" size=6 color=#000000> Tuning Parameter for ORTB2

In [90]:
# c =6, lambda = 0.0000024
c =  np.linspace(6, 6 , 1)
lambdas = np.linspace(0.0000020,  0.0000030, 10)

In [91]:
k=0
results  = pd.DataFrame(columns = ["impression","click","ctr","spend","cpm","cpc","c","lambdas"])     

for pram_c in c:
    for lamb in lambdas:
        pre_bidprices = Optimal_RealTime_Bidding_2(XG, pram_c, lamb)
        impression, click, ctr, spend, cpm, cpc = Evaluate(validation, pre_bidprices)
        results.loc[k]  = impression, click, ctr, spend, cpm, cpc, pram_c, lamb
        k = k+1

In [92]:
results.click.max()

148.0

In [95]:
best = results.loc[(results['click'] == 148)] 

In [96]:
best

,impression,click,ctr,spend,cpm,cpc,c,lambdas
3,156887.0,148.0,0.09434,6129515.0,39069.61699,41415.64,6.0,0.000002
4,154403.0,148.0,0.09585,5957744.0,38585.67515,40255.03,6.0,0.000002
5,152116.0,148.0,0.09729,5800513.0,38132.16887,39192.66,6.0,0.000003
6,149874.0,148.0,0.09875,5647812.0,37683.73434,38160.89,6.0,0.000003


<font face="微软雅黑" size=6 color=#000000> run on validation 

In [12]:
bidprices = Optimal_RealTime_Bidding_1(XG,6,0.000025)

In [20]:
Group_17 = pd.DataFrame(data = list(zip(validation_o["bidid"], bidprices)),
                               columns = ["bidid", "bidprice"])

In [22]:
Group_17.to_csv("Group_17.csv")